In [1]:
with open("import_modules.py") as f:
    exec(f.read())

All imported


In [3]:
df1 = pd.read_csv('books_new_modified_1348.csv')
#df2 = pd.read_csv('wiki_extended_divided_44654.csv')
df3 = pd.read_csv('essays_ext_3828.csv')
df_tot = pd.concat((df1, df3), ignore_index= True)

In [ ]:
url = 'http://localhost:8000/getExtraction'
headers = {'Content-type': 'text/plain'}    

In [ ]:
roman_reg = re.compile('(?<=^)M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})(?=$)')

def romanToInt(s):
    translations = {
        "I": 1,
        "V": 5,
        "X": 10,
        "L": 50,
        "C": 100,
        "D": 500,
        "M": 1000
    }
    number = 0
    s = s.replace("IV", "IIII").replace("IX", "VIIII")
    s = s.replace("XL", "XXXX").replace("XC", "LXXXX")
    s = s.replace("CD", "CCCC").replace("CM", "DCCCC")
    for char in s:
        number += translations[char]
    return number

def get_matches(text):
    tokenize = nltk.word_tokenize(text)
    dict = {}
    for token in tokenize:
        if roman_reg.match(token):
            dict[token] = romanToInt(token)
    return dict

def replace_matches(text):
    matches = get_matches(text)
    for key in matches:
        text = text.replace(key, str(matches[key]))
    return text

In [ ]:
#Pipeline with spacy to substitute pronouns 
# df = pd.read_csv('new_example.csv')
# nlp = spacy.load('en_core_web_sm')
# neuralcoref.add_to_pipe(nlp)
# d = {}
# for i in range(len(df)):
#     doc = nlp(df['text'][i])
#     d[df['title'][i]] = doc._.coref_resolved
# new_df = pd.DataFrame(list(d.items()),columns = ['title','text'])

In [ ]:
def sentence_tokenization(df):
    out = [sent_tokenize(par) for par in df.text]
    clean = []
    for par in out:
        for i in range(len(par)):
            if len(par[i]) > 5:
                clean.append(par)
    return clean

def relation_extraction(out):
    issues = 0
    #out = sentence_tokenization(df)
    rel_ext = []
    for par in out[:10]:
        for s in par:
            s = unidecode(s.replace(' ', ' ').replace('/', ' '))
            #s = s.replace('$', '').replace('. . .', '. ').replace('...', '. ')
            #s = replace_matches(s)
            s = s.replace(' the ', '')
            req = requests.request('POST', url, headers=headers, data = s.encode('utf-8'))
            try:
                js = req.json()
                rel_ext.extend(js)
            except:
                issues +=1
                #print('JSON issue')
    return issues, rel_ext

def create_rel_dataframe(rel):
    #issues, rel = relation_extraction(df)
    #print(rel)
    l  = []
    for r in rel:
        try:
            l.append((r['confidence'], r['sentence'], r['extraction']['arg1']['text'], r['extraction']['rel']['text'], 
                    r['extraction']['arg2s'][0]['text'], r['extraction']['negated'], r['extraction']['passive']))
        except IndexError:
            pass  
    l = list(dict.fromkeys(l))
    return pd.DataFrame(l, columns= ['confidence', 'sentence', 'arg1', 'rel', 'arg2', 'negated', 'passive'])

In [ ]:
out = sentence_tokenization(df_tot)
issues, rel = relation_extraction(out)
print(f'N. Issues: {issues}')
rel_df = create_rel_dataframe(rel)

In [ ]:
rel_df.head(10)

,confidence,sentence,arg1,rel,arg2,negated,passive
0,0.957243,Senator Charles Sumner of Massachusetts was an...,Senator Charles Sumner of Massachusetts,was,an avowed leader,False,True
1,0.964209,Senator Charles Sumner of Massachusetts was an...,Senator Charles Sumner of Massachusetts,was,an avowed Abolitionist,False,True
2,0.896876,"Aftersack of Lawrence, on May 21, 1856, he gav...",a bitter speech,called,The Crime Against Kansas,False,True
3,0.739567,"Aftersack of Lawrence, on May 21, 1856, he gav...",he,gave,"a bitter speech inSenate called ""The Crime Aga...",False,False
4,0.410514,"He blasted""murderous robbers from Missouri,"" c...",He,picked,fromdrunken vomit of an uneasy civilization,False,False
5,0.183517,"He blasted""murderous robbers from Missouri,"" c...",He,blasted calling,them,False,False
6,0.388787,"He blasted""murderous robbers from Missouri,"" c...",He,blasted,murderous robbers,False,False
7,0.410514,"He blasted""murderous robbers from Missouri,"" c...",He,picked,fromdrunken spew of an uneasy civilization,False,False
8,0.380895,"Part of this oratory was a bitter, personal ti...",Andrew Butler,[is] Senator [from],South Carolina,False,False
9,0.966586,"Part of this oratory was a bitter, personal ti...",Part of this oratory,was,"a bitter, personal tirade against South Caroli...",False,True
